In [ ]:
# 爬取商务部商品市场周度数据: https://cif.mofcom.gov.cn/cif/html/dataCenter/index.html?jgnfcprd
# 因为该网页点击按钮后虽然更新了数据，但不会跳转到新页面，这是动态页面，使用Beautiful Soup可能无法实现
# 所以使用selenium来实现动态爬取
# 环境前提：
# 1，安装selenium库
# 2，安装对应浏览器的驱动程序。selenium官方说明: https://www.selenium.dev/documentation/getting_started/installing_browser_drivers/
# 本例使用Chrome，chromedriver下载地址: https://chromedriver.storage.googleapis.com/index.html
# 版本需要与本机Chrome的版本一致。如何查看本机Chrome版本: 在Chrome地址栏中输入Chrome://version，第一行
# 将下载好的压缩包解压
# 将解压得到的chromedriver.exe移动到chrome.exe所在的目录，一般来说（例如本例）该位置是C:\Program Files (x86)\Google\Chrome\Application
# 最后，配置环境变量，右键此电脑（注意不是快捷方式）-属性-高级系统设置-环境变量-系统环境变量-Path-编辑-新建，添加好上条的目录位置
# 参考教程: https://blog.csdn.net/orange_xiang/article/details/82924296

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep

import pandas as pd 
import numpy as np
import time

In [ ]:
# driver是控制网页，模拟点击的入口，下文将经常使用
driver = webdriver.Chrome()

# 打开新网站时等待几秒让其完全呈现，但本例在同一网页上操作，可能没有作用
driver.implicitly_wait(10)

# 打开商务部价格网页
url = "https://cif.mofcom.gov.cn/cif/html/dataCenter/index.html?jgnfcprd"
driver.get(url)

In [ ]:
# 通过该组件的id，来定位位于左侧边栏的“周度数据”，然后点击
driver.find_element(by="id", value="li_jgnfcpzd").click()

In [ ]:
# 定位位于上方的日历，然后点击，为了调整时间范围
driver.find_element(by="id", value="nr5_start_date").click()

In [ ]:
# 日历下是一个iframe组件，driver在外部不可见其内部，所以要将工作目录转移为该iframe
driver.switch_to.frame(driver.find_element(by="xpath", value="/html/body/div[6]/iframe"))

# 包括下文，更新网页后紧接着插入一次sleep()来等待，避免代码向下执行时网页尚未更新好
# 如果在其他地方遇到类似问题，手动继续向下执行也容易解决
sleep(1)

In [ ]:
# 找到起始日期的输入栏
start_year = driver.find_element(by='xpath',value='/html/body/div/div[1]/div[4]/input')
# 直接退到1900年，这样最方便
start_year.clear()

# 提交
start_year.send_keys(Keys.ENTER)

In [ ]:
# 记得退回主工作目录
driver.switch_to.default_content()

In [ ]:
# 本模块对象区域是网页上方的商品按钮栏

# 每种商品的按钮都有一个不规则的id，于是建立一个空字典来存储这些信息
# 键是按钮的id，值是品种与类型的名称
# 品种：粳米、籼米、大米...
# 类型：粮油、肉类、禽蛋...
commodity_info_dict = {}

# 定位类型栏
category = driver.find_element(by='id', value='week_category')
# 收集类型栏下的每个分类
categories = category.find_elements(by='class name',value='yqpzl')

# 以下循环的结构由网页本身的结构所决定，注意移植时修改
for cat in categories:
    # 记录下类型的名称
    category_name = cat.find_element(by='class name', value='pzys.yqfont6').text
    # 收集该类型下的所有品种
    commodities = cat.find_element(by='class name', value='ygepz').find_elements(by='tag name', value='div')
    for com in commodities:
        # 记录下品种的名称、按钮id
        commodity_name = com.text
        commodity_id = com.get_attribute('id')
        # 将品种的按钮id、名称、类型添加到字典中
        commodity_info_dict.update({commodity_id : [commodity_name, category_name]})

In [ ]:
# 本模块对象区域主要是网页下方的价格表格

# 用作DataFrame的列表，独立存放每种商品
df_list = []

# 与上同理，以下代码的结构由网页本身的结构所决定，注意移植时修改

# 根据上一模块整理好的信息来依种类遍历
for key, value in commodity_info_dict.items():
    com_id = key
    com_name = value[0]
    com_cat = value[1]
    
    # 点击该商品的按钮
    driver.find_element(by="id", value=com_id).click()
    sleep(1)

    # 记录下表头head，用作colmuns
    table_head = driver.find_element(by='id', value='nr5_table_head').find_element(by='tag name', value='tr')
    head = table_head.text.split(' ')

    # 定位到表格主体
    table = driver.find_element(by='id', value='nr5_table')
    #收集表格的行
    table_content = table.find_elements(by='tag name', value='tr')

    # 用一个列表来存放原始数据
    temp_list = []
    # 遍历读行
    for tr in table_content:
        # 读行时会发现，每行的text是一整个字符串，有空格间隔，所以要split()一下
        temp_list.append(tr.text.split(' '))  
    # temp_list中每个元素也是列表，正好可以用来构造DataFrame，记得用之前记录好的表头作列名
    temp_df = pd.DataFrame(temp_list, columns=head)
    # 新增两列标明品种与类型
    temp_df['品种'] = com_name
    temp_df['类型'] = com_cat

    # 将DataFrame加入大列表中
    df_list.append(temp_df)

In [ ]:
# 拼接大列表中所有DataFrame
data = pd.concat(df_list, ignore_index=True)
data

In [ ]:
# 对列重新排序
data = data.reindex(columns=['时间','品种','类型','价格（元/公斤）','环比变化（元/公斤）','价格（元/升）','环比变化（元/升）','环比（%）'])
data

In [ ]:
# 简单检查是否完整
data['品种'].value_counts()

In [ ]:
# 牛奶的单位是升，这就是为什么之前要记录下每种商品的表头，不然单位会混淆
data[data['品种'] == '牛奶']

In [ ]:
# 自动添加上日期标记，导出数据
date = time.strftime("%Y-%m-%d", time.localtime()) 
data.to_csv('weekly_price_'+date, index=False)